# Description

This code collects the time resolved optical properties of Ta2NiSe5 as calculated from the equilibrium properties and time-resolved pump-probe optical spectroscopy, creates dataframes containing both the equilibrium and time-resolved data, and saves the resulting dataframes.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import scipy
import math
from scipy.signal import savgol_filter,boxcar
from pathlib import Path
from functools import reduce

import glob
import os

In [3]:
# Define constants

h = 6.62e-34 #Planck's constant
c = 3e8      #Speed of light
e = 1.6e-19  #Charge of the electron

### Define functions

In [4]:
def selectchars(x,Ref=True):
    
    """
    Function to select parts 
    of file names for the 
    purpose of labeling graphs.
    """
    
    if Ref:
        return(int(x[3:-6]))
    else:
        return(int(x[4:-6]))


def get_all_txt_names(folder):
    
    """
    Generate list of all required filenames
    """
    
    TFolder_R = os.path.join(folder,"dR")
    T_files_R = sorted([f for f in os.listdir(TFolder_R) if f.endswith(".txt")],
                         key=lambda f:selectchars(f,Ref=True))
    
    TFolder_S1 = os.path.join(folder,"dS1")
    T_files_S1 = sorted([f for f in os.listdir(TFolder_S1) if f.endswith(".txt")],
                         key=lambda f:selectchars(f,Ref=False))
    
    TFolder_S2 = os.path.join(folder,"dS2")
    T_files_S2 = sorted([f for f in os.listdir(TFolder_S2) if f.endswith(".txt")],
                         key=lambda f:selectchars(f,Ref=False))
    
    return T_files_R, T_files_S1, T_files_S2

def convert_to_wl(WN):
    
    """
    Convert wavenumbers to wavelenghts
    """
    
    wl = (10**7)/(WN)    
    return wl

def convert_to_En(wl):
    
    """
    Convert wavelength to energy
    """
    
    h = 6.62e-34
    c = 3e8
    e = 1.6e-19
    En = (h*c*10**9)/(e*wl)
    return En


def interp_stat_data(df,WN):
    """
    Interpolate the static data
    in the wavenumber of regions 
    where no data points were collected.
    """
    
    intfit=interp1d(df.iloc[:,0],df.iloc[:,1])
    data_array = intfit(WN.values)
    
    return pd.Series(data_array,name="stat")



def create_time_df(folder,files,stat_file,prop="dR"):
    """
    Create dataframe with equilibrium data and 
    a sequence of time steps.
    """
    
    sep = " " 
    
    if prop == "dR":
        Ref = True
    else:
        Ref = False
        
    df = pd.read_csv(os.path.join(folder,prop,files[0]),sep=sep,header=None,names=["WN","temp"],dtype='float32')
    df.drop("temp",axis=1,inplace=True)
    df["WN"] = df["WN"].apply(float)
    df["WL"] = df["WN"].apply(convert_to_wl)
    df["En"] = df["WL"].apply(convert_to_En)
    
    stat = pd.read_csv(stat_file, sep=",",header=None,names=["WN",prop]) 
    df["stat"] = interp_stat_data(stat,df["WN"])
    
    for f in files:
        
        tempdf = pd.read_csv(os.path.join(folder,prop,f),sep=sep,header=None,names=["WN",selectchars(f,Ref=Ref)],
                            dtype='float32')
        tempdf[selectchars(f,Ref=Ref)] = tempdf[selectchars(f,Ref=Ref)]+df["stat"]
        df = pd.merge(df,tempdf,on="WN",how="inner")
        
    return df     


def check_df_size(folder,prop="dR"):
    
    """
    Quickly check data folder sizes.
    Used in the troubleshooting process.
    """
    
    sep = " " 
    
    if prop == "dR":
        Ref = True
    else:
        Ref = False
    
    TFolder = os.path.join(folder,prop)
    files = sorted([f for f in os.listdir(TFolder) if f.endswith(".txt")],
                         key=lambda f:selectchars(f,Ref=Ref))
    
    Size = []

    sep = " " 
    
    for f in files:
        df = pd.read_csv(os.path.join(TFolder,f),sep=" ",header=None,names=["WN","temp"],
                            dtype='float32')
        Size.append(df.shape)
        
    return Size 
    

def create_all_df(folder):
    
    """
    Create all required dataframes using only 
    the fit to the data
    """
    
    R_stat_file = os.path.abspath(os.path.join(folder,os.pardir)) + "\R.csv"
    S1_stat_file = os.path.abspath(os.path.join(folder,os.pardir)) + "\S1_fit.csv"
    S2_stat_file = os.path.abspath(os.path.join(folder,os.pardir)) + "\S2_fit.csv"
    
    TimeRFiles,TimeS1Files,TimeS2Files = get_all_txt_names(folder)
    R_df = create_time_df(folder,TimeRFiles,R_stat_file)
    S1_df = create_time_df(folder,TimeS1Files,S1_stat_file,prop="dS1")
    S2_df = create_time_df(folder,TimeS2Files,S2_stat_file,prop="dS2")
    
    return R_df, S1_df, S2_df


def create_all_df_rawAndfit(folder):
    
    """
    Create all required dataframes including 
    both the raw data and the fits. 
    """
    
    R_stat_file = os.path.abspath(os.path.join(folder,os.pardir)) + "\R.csv"
    S1raw_stat_file = os.path.abspath(os.path.join(folder,os.pardir)) + "\S1_raw.csv"
    S1fit_stat_file = os.path.abspath(os.path.join(folder,os.pardir)) + "\S1_fit.csv"
    S2raw_stat_file = os.path.abspath(os.path.join(folder,os.pardir)) + "\S2_raw.csv"
    S2fit_stat_file = os.path.abspath(os.path.join(folder,os.pardir)) + "\S2_fit.csv"
    
    TimeRFiles,TimeS1Files,TimeS2Files = get_all_txt_names(folder)
    R_df = create_time_df(folder,TimeRFiles,R_stat_file)
    S1raw_df = create_time_df(folder,TimeS1Files,S1raw_stat_file,prop="dS1")
    S1fit_df = create_time_df(folder,TimeS1Files,S1fit_stat_file,prop="dS1")
    S2raw_df = create_time_df(folder,TimeS2Files,S2raw_stat_file,prop="dS2")
    S2fit_df = create_time_df(folder,TimeS2Files,S2fit_stat_file,prop="dS2")
    
    return R_df, S1raw_df, S1fit_df, S2raw_df, S2fit_df

### Define Data Folders

In [5]:
# Static data at Room Temperature
RT150fld = '../data/Unprocessed/RT/150uW/'
RT900fld = '../data/Unprocessed/RT/900uW/'
RT3500fld = '../data/Unprocessed/RT/3500uW/'
RT4000fld = '../data/Unprocessed/RT/4000uW/'
RT5000fld = '../data/Unprocessed/RT/5000uW/'
RT6000fld = '../data/Unprocessed/RT/6000uW/'

# Static data at 345 K
HT250fld = '../data/Unprocessed/HT/250uW/'
HT3000fld = '../data/Unprocessed/HT/3000uW/'

In [6]:
size = check_df_size(HT3000fld,"dS1")
print(size)

[(1000, 2), (1000, 2), (1000, 2), (1000, 2), (1000, 2), (1000, 2), (1000, 2), (1000, 2), (1000, 2), (1000, 2), (1000, 2), (1000, 2), (1000, 2), (1000, 2), (1000, 2), (1000, 2), (1000, 2), (1000, 2), (1000, 2), (1000, 2), (1000, 2), (1000, 2), (1000, 2), (1000, 2)]


### Construct dataframes with static and time resolved data

In [7]:
# Compare using raw or fit as static data

RT150_R, RT150_S1raw, RT150_S1fit, RT150_S2raw, RT150_S2fit = create_all_df_rawAndfit(RT150fld)
RT900_R, RT900_S1raw, RT900_S1fit, RT900_S2raw, RT900_S2fit = create_all_df_rawAndfit(RT900fld)
RT3500_R, RT3500_S1raw, RT3500_S1fit, RT3500_S2raw, RT3500_S2fit = create_all_df_rawAndfit(RT3500fld)
RT4000_R, RT4000_S1raw, RT4000_S1fit, RT4000_S2raw, RT4000_S2fit = create_all_df_rawAndfit(RT4000fld)
RT5000_R, RT5000_S1raw, RT5000_S1fit, RT5000_S2raw, RT5000_S2fit = create_all_df_rawAndfit(RT5000fld)
RT6000_R, RT6000_S1raw, RT6000_S1fit, RT6000_S2raw, RT6000_S2fit = create_all_df_rawAndfit(RT6000fld)

HT250_R, HT250_S1raw, HT250_S1fit, HT250_S2raw, HT250_S2fit = create_all_df_rawAndfit(HT250fld)
HT3000_R, HT3000_S1raw, HT3000_S1fit, HT3000_S2raw, HT3000_S2fit = create_all_df_rawAndfit(HT3000fld)

### Save dataframes

In [8]:
RT150_S1fit.to_csv('../data/Dataframes/RT150_S1.csv', index=False)
RT900_S1fit.to_csv('../data/Dataframes/RT900_S1.csv', index=False)
RT3500_S1fit.to_csv('../data/Dataframes/RT3500_S1.csv', index=False)
RT4000_S1fit.to_csv('../data/Dataframes/RT4000_S1.csv', index=False)
RT5000_S1fit.to_csv('../data/Dataframes/RT5000_S1.csv', index=False)
RT6000_S1fit.to_csv('../data/Dataframes/RT6000_S1.csv', index=False)

HT250_S1fit.to_csv('../data/Dataframes/HT250_S1.csv', index=False)
HT3000_S1fit.to_csv('../data/Dataframes/HT3000_S1.csv', index=False)